In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [2]:
class Model1 (nn.Module):
    def __init__(self):
        super(Model1,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(5,3),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(3,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.model(x)

In [3]:
df = pd.read_json("data3/normal_vpm.json",orient="records",lines=True)


In [4]:
list_data = df.to_numpy()
np.random.shuffle(list_data)

train_x = torch.Tensor(list_data[:10000,:-1])
train_Y = torch.Tensor(list_data[:10000,-1].reshape(-1,1))
train_data = TensorDataset(train_x,train_Y)
train_dataLoader = DataLoader(train_data,batch_size=128,shuffle=True)

test_x = torch.Tensor(list_data[10000:,:-1])
test_Y = torch.Tensor(list_data[10000:,-1].reshape(-1,1))
test_dataset = TensorDataset(test_x,test_Y)
test_dataLoader = DataLoader(test_dataset,batch_size=128,shuffle=True)


In [5]:
test_x[0:10]

tensor([[5.8511e-01, 9.0000e+00, 1.0170e-04, 2.0000e+01, 2.0000e+01],
        [9.7048e-03, 3.0000e+00, 2.0437e-03, 0.0000e+00, 0.0000e+00],
        [8.9707e-01, 1.6000e+01, 1.1792e-04, 2.0000e+01, 0.0000e+00],
        [2.4463e-05, 2.0000e+00, 5.2863e-01, 1.5000e+01, 1.5000e+01],
        [6.8987e-01, 4.0000e+00, 3.8335e-05, 1.5000e+01, 5.0000e+00],
        [3.1295e-05, 2.0000e+00, 4.1522e-01, 2.0000e+01, 2.0000e+01],
        [9.8330e-02, 2.0000e+00, 1.3448e-04, 0.0000e+00, 0.0000e+00],
        [6.9837e-01, 6.0000e+00, 5.6803e-05, 2.0000e+01, 1.5000e+01],
        [6.9569e-01, 9.0000e+00, 8.5532e-05, 2.0000e+01, 0.0000e+00],
        [6.9018e-01, 8.0000e+00, 7.6635e-05, 1.0000e+01, 5.0000e+00]])

In [6]:
model1 = Model1()
optimizer = optim.Adam(model1.parameters(),lr=0.06)
criterion = nn.MSELoss()

In [7]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [11]:
epochs = 20

model1.train()
for epoch in tqdm(range(epochs)):
    train_loss = []
    for x, Y in train_dataLoader:
        x = x.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        out = model1(x)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    print(f"Loss: {np.average(train_loss):.3f}")

 10%|█         | 2/20 [00:00<00:01, 14.29it/s]Loss: 0.188
Loss: 0.197
Loss: 0.195
 30%|███       | 6/20 [00:00<00:00, 14.30it/s]Loss: 0.185
Loss: 0.219
Loss: 0.184
 40%|████      | 8/20 [00:00<00:00, 14.28it/s]Loss: 0.185
Loss: 0.182
Loss: 0.182
 60%|██████    | 12/20 [00:00<00:00, 14.31it/s]Loss: 0.232
Loss: 0.182
Loss: 0.181
 80%|████████  | 16/20 [00:01<00:00, 14.64it/s]Loss: 0.183
Loss: 0.175
Loss: 0.176
Loss: 0.175
100%|██████████| 20/20 [00:01<00:00, 14.55it/s]Loss: 0.173
Loss: 0.193
Loss: 0.181
Loss: 0.176



In [12]:
correct = 0
for x, Y in test_dataLoader:
    model1.eval()
    x = x.to(device)
    Y = Y.cpu()
    out=model1(x)
    out.cpu()
    for i,o in enumerate(out):
        if o * Y[i] > 0:
            correct += 1
print(f'Corectly predicted: {correct} out of {len(test_dataset)}\nTest accuracy: {correct/len(test_dataset):.2%}')


Corectly predicted: 9735 out of 10348
Test accuracy: 94.08%


In [10]:
model1.cpu()
print(model1(torch.Tensor(np.array([[0.005,4,0.00003,0,0],[0.005,4,0.00003,0,5],[0.005,4,0.00003,0,10],[0.005,4,0.00003,0,15],[0.005,4,0.00003,0,20]]))))
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))

tensor([[0.9407],
        [0.9727],
        [0.9876],
        [0.9944],
        [0.9975]], grad_fn=<TanhBackward>)
